<a href="https://colab.research.google.com/github/ahelmasri87/-AAI614_elmasri/blob/main/FWA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import traceback
import os

def get_user_input():
    root = tk.Tk()
    root.withdraw()  # Close the root window
    file_path = filedialog.askopenfilename(title="Select file", filetypes=[("Excel files", "*.xlsx *.xls")])
    if not file_path:
        print("No file selected. Exiting.")
        return None, None, False, False, False, False

    print("Welcome to FWA analysis (this version is still under testing and development):")
    print("Do you want to:")
    print("1. Search by keywords")
    print("2. Temperature Analysis")
    print("3. Vital Sign Analysis")
    print("4. Unique Data Frequency")
    print("5. Duplicate Vital Sign Claims")

    choice = input("Enter your choice: ")

    if choice == '1':
        keywords = input("Please enter the keywords separated by commas: ").split(',')
        keywords = [keyword.strip() for keyword in keywords]  # Remove any extra spaces
        return file_path, keywords, False, False, False, False
    elif choice == '2':
        return file_path, None, True, False, False, False
    elif choice == '3':
        return file_path, None, False, True, False, False
    elif choice == '4':
        return file_path, None, False, False, True, False
    elif choice == '5':
        return file_path, None, False, False, False, True
    else:
        print("Invalid choice. Exiting.")
        return None, None, False, False, False, False

def search_keywords_with_context(file_path, keywords):
    print("Searching keywords...")
    df = pd.read_excel(file_path)

    keywords = [keyword.lower() for keyword in keywords]

    results = []
    for index, row in df.iterrows():
        combined_text = ' '.join(row.astype(str).str.lower())

        found_keywords = [keyword for keyword in keywords if keyword in combined_text]
        if found_keywords:
            result = {
                'NphiesClaimID': row.get('NphiesClaimID', 'N/A'),
                'ValueString': row.get('ValueString', 'N/A'),
                'found_keywords': ', '.join(found_keywords)
            }
            results.append(result)

    print(f"Found {len(results)} matching rows.")
    return pd.DataFrame(results)

def analyze_temperature_data(file_path):
    print("Analyzing temperature data...")
    df = pd.read_excel(file_path)

    fever_keywords = ["fever", "fvr", "fevr", "fver"]

    fever_rows = df[df['ValueString'].str.contains('|'.join(fever_keywords), case=False, na=False)]
    fever_ids = fever_rows['NphiesClaimID'].unique()
    category_temperature_rows = df[df['Category'].str.lower() == 'temperature']
    temperature_with_fever_rows = category_temperature_rows[category_temperature_rows['NphiesClaimID'].isin(fever_ids)]
    results = temperature_with_fever_rows[temperature_with_fever_rows['ValueQuantity'] < 38]

    chief_complaint = df[df['Category'].str.lower() == 'chief-complaint'][['NphiesClaimID', 'ValueString']]
    results = results.merge(chief_complaint, on='NphiesClaimID', suffixes=('', '_chief_complaint'))

    columns_to_drop = ['ValueReference', 'Reason', 'AttachContentType', 'AttachLanguage',
                       'AttachmentTitle', 'AttachmentURL', 'AttachmentSize', 'TimingEndDate', 'FilePath']
    results = results.drop(columns=columns_to_drop, errors='ignore')

    print(f"Found {len(results)} rows with temperature data below 38.")
    return results

def analyze_vital_sign_data(file_path):
    print("Analyzing vital sign data...")
    df = pd.read_excel(file_path)

    def detect_anomalies(row, normal_ranges, chief_complaint):
        category = row['Category']
        value = row['ValueQuantity']
        if pd.isna(value):
            return False, None
        if category == 'temperature' and 'fever' in str(chief_complaint).lower():
            return False, None
        if category in normal_ranges:
            lower, upper = normal_ranges[category]
            if value < lower or value > upper:
                return True, f"{category} value {value} is outside the normal range ({lower}-{upper})"
        return False, None

    categories_of_interest = ['pulse', 'respiratory-rate', 'temperature', 'vital-sign-diastolic', 'vital-sign-systolic']
    filtered_data = df[df['Category'].isin(categories_of_interest)].copy()
    chief_complaints = df[df['Category'] == 'chief-complaint'][['NphiesClaimID', 'ValueString']]
    chief_complaints.columns = ['NphiesClaimID', 'chief-complaint']
    filtered_data = filtered_data.merge(chief_complaints, on='NphiesClaimID', how='left')

    normal_ranges = {
        'pulse': (60, 100),
        'respiratory-rate': (12, 20),
        'temperature': (36.1, 37.2),
        'vital-sign-diastolic': (60, 80),
        'vital-sign-systolic': (90, 120)
    }

    anomalies_list = filtered_data.apply(lambda row: detect_anomalies(row, normal_ranges, row['chief-complaint']), axis=1)
    filtered_data['Anomaly'], filtered_data['AnomalyReason'] = zip(*anomalies_list)
    anomalies = filtered_data[filtered_data['Anomaly'] == True]

    days_supply = df[df['Category'] == 'days-supply'][['NphiesClaimID', 'ValueQuantity']]
    days_supply.columns = ['NphiesClaimID', 'days-supply']
    detailed_anomalies = anomalies.merge(days_supply, on='NphiesClaimID', how='left')

    output_file = 'anomalies_output.xlsx'
    detailed_anomalies.to_excel(output_file, index=False)
    print(f"Anomalies have been exported to {output_file}")
    return detailed_anomalies

def calculate_unique_data_frequency(file_path):
    print("Calculating unique data frequency...")
    df = pd.read_excel(file_path)

    categories_of_interest = ['pulse', 'respiratory-rate', 'temperature', 'vital-sign-diastolic', 'vital-sign-systolic']
    filtered_data = df[df['Category'].isin(categories_of_interest)].copy()

    frequency_data = {}
    for category in categories_of_interest:
        category_data = filtered_data[filtered_data['Category'] == category]
        frequency_counts = category_data['ValueQuantity'].value_counts().reset_index()
        frequency_counts.columns = ['Value', 'Frequency']
        frequency_data[category] = frequency_counts

    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    output_file_base = os.path.join(desktop, 'unique_data_frequency.xlsx')
    output_file = output_file_base

    # If the file already exists, add a sequence number to it
    sequence = 1
    while os.path.exists(output_file):
        output_file = os.path.join(desktop, f'unique_data_frequency_{sequence}.xlsx')
        sequence += 1

    with pd.ExcelWriter(output_file) as writer:
        for category, data in frequency_data.items():
            data.to_excel(writer, sheet_name=category, index=False)
    print(f"Unique data frequency has been exported to {output_file}")
    return frequency_data

def find_duplicate_vital_signs(file_path):
    try:
        print("Finding duplicate vital sign claims...")
        df = pd.read_excel(file_path)
        print("Data loaded successfully.")

        categories_of_interest = ['pulse', 'vital-sign-systolic', 'vital-sign-diastolic', 'temperature', 'respiratory-rate']
        filtered_data = df[df['Category'].isin(categories_of_interest)].copy()
        print("Filtered data based on categories of interest.")

        pivot_table = filtered_data.pivot_table(index='NphiesClaimID', columns='Category', values='ValueQuantity', aggfunc='first')
        duplicated = pivot_table[pivot_table.duplicated(keep=False)]

        if duplicated.empty:
            print("No duplicate vital signs found.")
            return pd.DataFrame()  # Return an empty DataFrame if no duplicates are found

        print("Pivot table of duplicates:")
        print(duplicated)

        # Merge with the original dataframe to get full details
        duplicated_ids = duplicated.index
        duplicate_details = filtered_data[filtered_data['NphiesClaimID'].isin(duplicated_ids)]

        print("Duplicate details:")
        print(duplicate_details)

        # Ensure the columns in duplicate_details match those in categories_of_interest
        available_categories = [category for category in categories_of_interest if category in pivot_table.columns]
        for category in available_categories:
            if category not in duplicate_details.columns:
                duplicate_details.loc[:, category] = None

        # Convert columns to float64 to match the types in duplicate_counts
        for category in available_categories:
            duplicate_details.loc[:, category] = duplicate_details.loc[:, category].astype(float)

                # Group by the available vital sign values and count occurrences
        duplicate_counts = pivot_table[pivot_table.index.isin(duplicated_ids)].groupby(available_categories).size().reset_index(name='Count')

        print("Duplicate counts:")
        print(duplicate_counts)

        # Create a dictionary to hold the DataFrames for each set of identical vital signs
        duplicates_dict = {}
        for _, row in duplicate_counts.iterrows():
            condition = (pivot_table[available_categories] == row[available_categories].values).all(axis=1)
            matching_ids = pivot_table[condition].index
            duplicates_dict[f'Duplicates_set_{row["Count"]}_count'] = pivot_table.loc[matching_ids].reset_index()

        # Define the output file path
        desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
        output_file_base = os.path.join(desktop, 'detailed_duplicate_vital_signs.xlsx')
        output_file = output_file_base

        # If the file already exists, add a sequence number to it
        sequence = 1
        while os.path.exists(output_file):
            output_file = os.path.join(desktop, f'detailed_duplicate_vital_signs_{sequence}.xlsx')
            sequence += 1

        # Write each DataFrame to a separate sheet in the Excel file
        with pd.ExcelWriter(output_file) as writer:
            for sheet_name, data in duplicates_dict.items():
                data.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Detailed duplicate vital signs have been exported to {output_file}")
        return duplicate_details
    except Exception as e:
        print(f"An error occurred: {e}")
        print(traceback.format_exc())
        return pd.DataFrame()

def export_results_to_excel(results, output_file_path):
    try:
        results.to_excel(output_file_path, index=False)
        print(f"Results have been exported to {output_file_path}")
    except Exception as e:
        print(f"An error occurred while exporting to Excel: {e}")
        print(traceback.format_exc())

def main():
    while True:
        file_path, keywords, start_with_option_5, start_with_option_vital, start_with_option_frequency, start_with_option_duplicates = get_user_input()
        if not file_path:
            print("No file path provided. Exiting.")
            break

        results = pd.DataFrame()

        if start_with_option_5:
            results = analyze_temperature_data(file_path)
        elif start_with_option_vital:
            results = analyze_vital_sign_data(file_path)
        elif start_with_option_frequency:
            results = calculate_unique_data_frequency(file_path)
        elif start_with_option_duplicates:
            results = find_duplicate_vital_signs(file_path)
        else:
            results = search_keywords_with_context(file_path, keywords)

        temperature_results = pd.DataFrame()

        while True:
            print("\nOptions:")
            print("1. Reset and upload new file")
            print("2. Make a new keyword search on the uploaded Excel")
            print("3. Export the outcome to Excel")
            print("4. Exit the module")
            print("5. Analyze temperature data")
            print("6. Analyze vital sign data")
            print("7. Calculate unique data frequency")
            print("8. Find duplicate vital sign claims")
            choice = input("Enter your choice: ")

            if choice == '1':
                break  # Break the loop to reset and upload a new file
            elif choice == '2':
                keywords = input("Please enter the keywords separated by commas: ").split(',')
                keywords = [keyword.strip() for keyword in keywords]
                results = search_keywords_with_context(file_path, keywords)
            elif choice == '3':
                if not results.empty or not temperature_results.empty:
                    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
                    output_file_base = os.path.join(desktop, 'results.xlsx')
                    output_file_path = output_file_base

                    # If the file already exists, add a sequence number to it
                    sequence = 1
                    while os.path.exists(output_file_path):
                        output_file_path = os.path.join(desktop, f'results_{sequence}.xlsx')
                        sequence += 1

                    if output_file_path:
                        if not temperature_results.empty:
                            export_results_to_excel(temperature_results, output_file_path)
                        else:
                            export_results_to_excel(results, output_file_path)
                    else:
                        print("No file selected for saving.")
                else:
                    print("No results to export.")
            elif choice == '4':
                print("Exiting the module.")
                return
            elif choice == '5':
                temperature_results = analyze_temperature_data(file_path)
                if temperature_results.empty:
                    print("No temperature data to export.")
                else:
                    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
                    output_file_base = os.path.join(desktop, 'temperature_results.xlsx')
                    output_file_path = output_file_base

                    # If the file already exists, add a sequence number to it
                    sequence = 1
                    while os.path.exists(output_file_path):
                        output_file_path = os.path.join(desktop, f'temperature_results_{sequence}.xlsx')
                        sequence += 1

                    if output_file_path:
                        export_results_to_excel(temperature_results, output_file_path)
                    else:
                        print("No file selected for saving.")
            elif choice == '6':
                vital_sign_results = analyze_vital_sign_data(file_path)
                if vital_sign_results.empty:
                    print("No vital sign data to export.")
                else:
                    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
                    output_file_base = os.path.join(desktop, 'vital_sign_results.xlsx')
                    output_file_path = output_file_base

                    # If the file already exists, add a sequence number to it
                    sequence = 1
                    while os.path.exists(output_file_path):
                        output_file_path = os.path.join(desktop, f'vital_sign_results_{sequence}.xlsx')
                        sequence += 1

                    if output_file_path:
                        export_results_to_excel(vital_sign_results, output_file_path)
                    else:
                        print("No file selected for saving.")
            elif choice == '7':
                frequency_results = calculate_unique_data_frequency(file_path)
                desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
                output_file_base = os.path.join(desktop, 'frequency_results.xlsx')
                output_file_path = output_file_base

                # If the file already exists, add a sequence number to it
                sequence = 1
                while os.path.exists(output_file_path):
                    output_file_path = os.path.join(desktop, f'frequency_results_{sequence}.xlsx')
                    sequence += 1

                if output_file_path:
                    with pd.ExcelWriter(output_file_path) as writer:
                        for category, data in frequency_results.items():
                            data.to_excel(writer, sheet_name=category, index=False)
                    print(f"Frequency data has been exported to {output_file_path}")
                else:
                    print("No file selected for saving.")
            elif choice == '8':
                duplicate_results = find_duplicate_vital_signs(file_path)
                if duplicate_results.empty:
                    print("No duplicate vital sign claims found.")
                else:
                    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
                    output_file_base = os.path.join(desktop, 'duplicate_vital_signs_results.xlsx')
                    output_file_path = output_file_base

                    # If the file already exists, add a sequence number to it
                    sequence = 1
                    while os.path.exists(output_file_path):
                        output_file_path = os.path.join(desktop, f'duplicate_vital_signs_results_{sequence}.xlsx')
                        sequence += 1

                    if output_file_path:
                        export_results_to_excel(duplicate_results, output_file_path)
                    else:
                        print("No file selected for saving.")
            else:
                print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


